In [ ]:
from skimage import io
from skimage.color import rgb2gray
from skimage.metrics import structural_similarity as ssim
from skimage.transform import resize
import cv2
import os

# Base directory where your images are stored
base_dir = "/workspaces/CSDNA/media/"

# Function to preprocess images (resizing and grayscaling)
def preprocess_image(image_path):
    # Load the image
    image = io.imread(image_path)
    # If the image has an alpha channel, remove it
    if image.shape[-1] == 4:
        image = image[..., :3]
    # Convert the image to grayscale
    image_gray = rgb2gray(image)
    # Resize the image to a fixed size
    image_resized = resize(image_gray, (256, 256), anti_aliasing=True)
    return image_resized

# Reference image (assuming the first image is the reference)
reference_image_path = "/workspaces/CSDNA/Testing code/Test 2/density.png"
reference_image = preprocess_image(reference_image_path)

# Dictionary to store SSIM scores
ssim_scores = {}

# Loop through image files and compute SSIM
for i in range(1, 100):
    image_path = os.path.join(base_dir, f"group_{i}.png")
    if os.path.exists(image_path):  # Check if the image file exists
        # Preprocess the image
        current_image = preprocess_image(image_path)
        # Compute SSIM with the reference image
        score = ssim(reference_image, current_image, data_range=current_image.max() - current_image.min())
        ssim_scores[f"group_{i}"] = score

# Find the image with the highest SSIM score
best_image = max(ssim_scores, key=ssim_scores.get)
print(f"The best matching image is {best_image} with an SSIM score of {ssim_scores[best_image]}")

#finding the image with the least SSIM score
worst_image = min(ssim_scores, key=ssim_scores.get)
print(f"The worst matching image is {worst_image} with an SSIM score of {ssim_scores[worst_image]}")

#generating a plot of the SSIM scores of number of images within a range and also generating the statistics for mean median and standard deviation
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Create a pandas DataFrame from the dictionary of SSIM scores
df = pd.DataFrame(list(ssim_scores.items()), columns=['Image', 'SSIM'])

# Plot the SSIM scores
plt.figure(figsize=(10, 6))
sns.histplot(df['SSIM'], kde=True)
plt.title('SSIM Scores')
plt.xlabel('SSIM')
plt.ylabel('Count')
plt.show()

#generating the list of groups with more than 0.550 SSIM scores and their specific group number along with SSIM score

# Filter the DataFrame to include only the rows with SSIM > 0.550
df_filtered = df[df['SSIM'] > 0.590]
print(df_filtered)

#finding if there is 14 in df_filtered
if 'group_14' in df_filtered.values:
    print("Yes, group_14 is in the filtered dataframe")


In [ ]:
import open_clip

# Load the pretrained CLIP model
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16-plus-240', pretrained="laion400m_e32")


# Base directory where your images are stored
base_dir = "/workspaces/CSDNA/media/"

# Define a function to encode images
def imageEncoder(img_path):
    img = Image.open(img_path)
    img = img.convert('RGB') if img.mode != 'RGB' else img
    img = preprocess(img).unsqueeze(0).to(device)
    return model.encode_image(img)

# Function to calculate the cosine similarity score between two images
ideal_image_path = "/workspaces/CSDNA/Testing code/Test 2/density.png"
other_image_paths = [os.path.join(base_dir, f"group_{i}.png") for i in range(1, 100)]

for other_image_path in other_image_paths:
    similarity_score = generateScore(ideal_image_path, other_image_path)
    print(f"Similarity Score for {other_image_path}: {similarity_score:.2f}")
